In [ ]:
!pip install textacy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.1 MB/s eta 0:00:00


In [ ]:
!pip install langdetect --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import pickle
import ast

import spacy
import pandas as pd
import numpy as np
import textacy
from textacy import text_stats
from IPython.display import clear_output
from google.colab import files
from langdetect import detect
from langdetect import detect_langs
from tqdm import tqdm

In [ ]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DEPENDENCY_TAGS = ['ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']

In [ ]:
!python -m spacy download en_core_web_lg --quiet

2023-09-05 17:47:55.933940: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


## Auxiliary Functions

In [ ]:
def remove_short_texts(df, min_len, max_len):
  df['text_length'] = df['text'].str.len()
  df = df.drop(df.loc[df['text_length'] < min_len].index)
  return df.drop(df.loc[df['text_length'] > max_len].index)

# Download and Preprocess Text Datasets

## Narations

In [ ]:
!kaggle datasets download michaelrussell4/10000-books-and-their-genres-standardized
!unzip 10000-books-and-their-genres-standardized.zip

 99% 1.29G/1.30G [00:16<00:00, 66.0MB/s]
100% 1.30G/1.30G [00:16<00:00, 85.8MB/s]
Archive:  10000-books-and-their-genres-standardized.zip
  inflating: books_and_genres.csv    


In [ ]:
data_books = pd.read_csv('books_and_genres.csv')
data_books.drop(columns=['Unnamed: 0'], inplace=True)
data_books.dropna(subset=['text'], inplace = True)
data_books.reset_index(drop=True, inplace = True)

for genre in ['poetry', 'religion', 'philosophy', 'historical', 'classics', 'humor', 'politics', 'short-stories', 'read-for-school', 'roman', 'non-fiction', 'history', '21st-century', 'literature', 'novels', 'fiction', 'horror', 'fantasy', 'speculative-fiction', 'science-fiction', 'supernatural', 'paranormal', '20th-century', 'dark', 'adult', 'mystery', 'adult-fiction', 'feminism', 'memoir', 'family', 'literary-fiction', 'college', 'high-school', 'american', 'mythology', 'biography', 'school', 'contemporary', 'historical-fiction', 'essays', 'coming-of-age', 'vampires', 'romance', 'novella', 'lgbt', 'thriller', 'unfinished', 'mystery-thriller', 'suspense', 'crime', 'science', 'theology', 'travel', 'adventure', 'relationships', 'cookbooks', 'reference', 'childrens', 'graphic-novels', 'comics', 'picture-books', 'action', 'magic', 'urban-fantasy', 'love', 'young-adult', 'paranormal-romance', 'teen', 'plays', 'dystopia', 'christian', 'education', 'psychology', 'economics', 'spirituality', 'writing', 'drama', 'art', 'middle-grade', 'christmas', 'chick-lit', 'new-adult', 'war', 'business', 'sociology', 'amazon', 'comedy', 'manga', 'erotica', 'animals', 'modern', 'self-help', 'realistic-fiction', 'historical-romance', 'sports', 'contemporary-romance', 'romantic-suspense', 'music', 'death', 'bdsm']:
  data_books[genre] = 0

for i in range(len(data_books)):
  for genre in ast.literal_eval(data_books.iloc[i].genres):
    data_books.at[i, genre] = 1

data_books['lang'] = data_books.text.apply(detect)
data_books.drop(data_books[data_books.lang != 'en'].index, inplace=True)
data_books.reset_index(drop=True, inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  del sys.path[0]


In [ ]:
data_books_list = [data_books.loc[i:i+106-1,:] for i in range(0, len(data_books), 106)]

## Short Stories

In [ ]:
!kaggle datasets download ratthachat/writing-prompts
!unzip writing-prompts.zip

 94% 349M/370M [00:03<00:00, 94.2MB/s]
100% 370M/370M [00:03<00:00, 118MB/s] 
Archive:  writing-prompts.zip
  inflating: writingPrompts/README   
  inflating: writingPrompts/test.wp_source  
  inflating: writingPrompts/test.wp_target  
  inflating: writingPrompts/train.wp_source  
  inflating: writingPrompts/train.wp_target  
  inflating: writingPrompts/valid.wp_source  
  inflating: writingPrompts/valid.wp_target  


In [ ]:
short_story_dict = {
    'prompt'  : [],
    'text'   : [],
}

#aux_train_target = open('writingPrompts/train.wp_target', 'r').read().split('\n')
aux_test_target = open('writingPrompts/test.wp_target', 'r').read().split('\n')
aux_valid_target = open('writingPrompts/valid.wp_target', 'r').read().split('\n')

#aux_train_source = open('writingPrompts/train.wp_source', 'r').read().split('\n')
aux_test_source = open('writingPrompts/test.wp_source', 'r').read().split('\n')
aux_valid_source = open('writingPrompts/valid.wp_source', 'r').read().split('\n')

# for i in range(len(aux_train_target)):
#   if aux_train_target[i] != '' and aux_train_source[i] != '':
#     short_story_dict['prompt'].append(aux_train_source[i])
#     short_story_dict['text'].append(aux_train_target[i].replace('<newline>', '\n'))

for i in range(len(aux_test_target)):
  if aux_test_target[i] != '' and aux_test_source[i] != '':
    short_story_dict['prompt'].append(aux_test_source[i])
    short_story_dict['text'].append(aux_test_target[i].replace('<newline>', '\n'))

for i in range(len(aux_valid_target)):
  if aux_valid_target[i] != '' and aux_valid_source[i] != '':
    short_story_dict['prompt'].append(aux_valid_source[i])
    short_story_dict['text'].append(aux_valid_target[i].replace('<newline>', '\n'))

data_shortstory = pd.DataFrame(short_story_dict)

In [ ]:
data_shortstory_list = [data_shortstory.loc[i:i+30336-1,:] for i in range(0, len(data_shortstory), 30336)]

## News

In [ ]:
!kaggle datasets download snapcrack/all-the-news
!unzip all-the-news.zip

In [ ]:
data_news = pd.concat([pd.read_csv('articles1.csv'), pd.read_csv('articles2.csv'), pd.read_csv('articles3.csv')], axis=0)
data_news.rename(columns = {'content':'text'}, inplace = True)
data_news.dropna(subset=['text'], inplace = True)
data_news.drop(columns=['Unnamed: 0'], inplace = True)
data_news = remove_short_texts(data_news, min_len=1500, max_len=10000)
data_news.reset_index(drop=True, inplace = True)

## Poems

In [ ]:
!kaggle datasets download tgdivy/poetry-foundation-poems
!unzip poetry-foundation-poems.zip

100% 8.88M/8.88M [00:00<00:00, 18.6MB/s]
100% 8.88M/8.88M [00:00<00:00, 18.7MB/s]
Archive:  poetry-foundation-poems.zip
  inflating: PoetryFoundationData.csv  


In [ ]:
data_poems = pd.read_csv('PoetryFoundationData.csv')
data_poems.rename(columns = {'Poem':'text'}, inplace = True)
data_poems.dropna(subset=['text'], inplace = True)
data_poems = remove_short_texts(data_poems, 300, 2500)
data_poems.reset_index(drop=True, inplace = True)

## Recipes

In [ ]:
!kaggle datasets download shuyangli94/foodcom-recipes-with-search-terms-and-tags
!unzip foodcom-recipes-with-search-terms-and-tags.zip

 98% 227M/231M [00:09<00:00, 22.2MB/s]
100% 231M/231M [00:09<00:00, 26.2MB/s]
Archive:  foodcom-recipes-with-search-terms-and-tags.zip
  inflating: recipes_w_search_terms.csv  


In [ ]:
data_recipes = pd.read_csv('recipes_w_search_terms.csv')
data_recipes['steps'] = data_recipes['steps'].apply(lambda x: ' '.join(ast.literal_eval(x)))
data_recipes.rename(columns = {'steps':'text'}, inplace = True)
data_recipes.dropna(subset=['text'], inplace = True)
data_recipes = remove_short_texts(data_recipes, min_len=200, max_len=2000)
data_recipes.reset_index(drop=True, inplace = True)
data_recipes = data_recipes.sample(n=100000, random_state=42)
data_recipes.reset_index(drop=True, inplace = True)

## Presentations

In [ ]:
!kaggle datasets download thegupta/ted-talk
!unzip ted-talk.zip

 34% 9.00M/26.8M [00:00<00:00, 59.8MB/s]
100% 26.8M/26.8M [00:00<00:00, 129MB/s] 
Archive:  ted-talk.zip
  inflating: TED_Talk.csv            


In [ ]:
data_ted = pd.read_csv('TED_Talk.csv')
data_ted.rename(columns = {'transcript':'text'}, inplace = True)
data_ted.dropna(subset=['text'], inplace = True)
data_ted['text'] = data_ted['text'].apply(lambda x: x.replace('(Audience)', ' ').replace('(Laughter)', ' ').replace('(Applause)', ' ').replace('(Music)', ' '))
data_ted = remove_short_texts(data_ted, min_len=2000, max_len=20000)
data_ted.reset_index(drop=True, inplace = True)

## Movie Dialogs

In [ ]:
!kaggle datasets download Cornell-University/movie-dialog-corpus
!unzip movie-dialog-corpus.zip

 56% 5.00M/8.91M [00:00<00:00, 22.4MB/s]
100% 8.91M/8.91M [00:00<00:00, 37.4MB/s]
Archive:  movie-dialog-corpus.zip
  inflating: README.txt              
  inflating: movie_characters_metadata.tsv  
  inflating: movie_conversations.tsv  
  inflating: movie_lines.tsv         
  inflating: movie_titles_metadata.tsv  
  inflating: raw_script_urls.tsv     


In [ ]:
def get_text_conversation(x):

  # convert string to list of conversation ids
  x = x[1:-1].split(' ')

  # remove \' before and after each line id
  x = [x[i][1:-1] for i in range(len(x))]

  # replace line id with text
  aux_x = []
  for line_id in x:

    if line_id not in aux_data_lines.index:
      aux_x = []
      break

    aux_x.append(aux_data_lines.loc[line_id].text)

  return '\n'.join(aux_x)

In [ ]:
aux_data_lines = pd.read_csv(
    'movie_lines.tsv',
    encoding='utf-8-sig',
    sep='\t',
    header = None,
    names = ['lineID', 'charID', 'movieID', 'charName', 'text'],
    index_col=['lineID']
)
aux_data_lines.dropna(subset=['text'], inplace = True)

data_conversations = pd.read_csv(
    'movie_conversations.tsv',
    sep='\t',
    encoding='ISO-8859-2',
    names = ['charID_1', 'charID_2', 'movieID', 'conversation']
)
data_conversations.dropna(subset=['conversation'], inplace = True)

# convert line ids to conversation
data_conversations['conversation'] = data_conversations['conversation'].apply(lambda x: get_text_conversation(x))

data_conversations.rename(columns = {'conversation':'text'}, inplace = True)
data_conversations = remove_short_texts(data_conversations, min_len=300, max_len=1500)
data_conversations.reset_index(drop=True, inplace = True)

## Wine Reviews

In [ ]:
!kaggle datasets download zynicide/wine-reviews
!unzip wine-reviews.zip

 92% 47.0M/50.9M [00:00<00:00, 260MB/s]
100% 50.9M/50.9M [00:00<00:00, 247MB/s]
Archive:  wine-reviews.zip
  inflating: winemag-data-130k-v2.csv  
  inflating: winemag-data-130k-v2.json  
  inflating: winemag-data_first150k.csv  


In [ ]:
data_wine = pd.read_csv('winemag-data_first150k.csv')
data_wine.rename(columns = {'description':'text'}, inplace = True)
data_wine.dropna(subset=['text'], inplace = True)
data_wine.drop(columns=['Unnamed: 0'], inplace = True)
data_wine = remove_short_texts(data_wine, min_len=200, max_len=500)
data_wine.reset_index(drop=True, inplace = True)

## Religious Texts

In [ ]:
!kaggle datasets download tentotheminus9/religious-and-philosophical-texts
!unzip religious-and-philosophical-texts.zip

  0% 0.00/2.57M [00:00<?, ?B/s]
100% 2.57M/2.57M [00:00<00:00, 120MB/s]
Archive:  religious-and-philosophical-texts.zip
  inflating: 35895-0.txt             
  inflating: pg10.txt                
  inflating: pg17.txt                
  inflating: pg2680.txt              
  inflating: pg2800.txt              


In [ ]:
religion_dict = {
    'author'  : [],
    'title'   : [],
    'text'    : []
}

In [ ]:
james_bible = open('pg10.txt', 'r').read()
start_idx = james_bible.find('*** START OF THIS PROJECT GUTENBERG EBOOK THE KING JAMES BIBLE ***\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n') + len('*** START OF THIS PROJECT GUTENBERG EBOOK THE KING JAMES BIBLE ***\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
final_idx = james_bible.find('\n\n\n\n\n\n\n\n\n\n\nEnd of the Project Gutenberg EBook of The King James Bible')
james_bible = james_bible[start_idx:final_idx]

works = ['The First Book of Moses:  Called Genesis', 'The Second Book of Moses:  Called Exodus', 'The Third Book of Moses:  Called Leviticus', 'The Fourth Book of Moses:  Called Numbers', 'The Fifth Book of Moses:  Called Deuteronomy', 'The Book of Joshua', 'The Book of Judges']
fillers = []
for i in range(50, 0, -1):
  for j in range(50, 0, -1):
    fillers.append(str(i) + ':' + str(j) + ' ')

b = []
for i in range(len(works)):
  if i < len(works) - 1:
    start = james_bible.find(works[i])
    end = james_bible.find(works[i+1])
    b.append(james_bible[start:end])
  else:
    start = james_bible.find(works[i])
    b.append(james_bible[start:])

for i in range(len(b)):
  for f in fillers:
    b[i] = b[i].replace(f, '')
  religion_dict['author'].append(None)
  religion_dict['title'].append(works[i])
  religion_dict['text'].append(b[i])

In [ ]:
mormon_book = open('pg17.txt', 'r').read()
start_idx = mormon_book.find('*** START OF THIS PROJECT GUTENBERG EBOOK THE BOOK OF MORMON ***\n\n\n\n\n\n\n\n\n\n\n') + len('*** START OF THIS PROJECT GUTENBERG EBOOK THE BOOK OF MORMON ***\n\n\n\n\n\n\n\n\n\n\n')
final_idx = mormon_book.find('\n\n\n\nEnd of the Project Gutenberg EBook of The Book Of Mormon')
mormon_book = mormon_book[start_idx:final_idx]

works = ['THE TESTIMONY OF THREE WITNESSES', 'THE TESTIMONY OF EIGHT WITNESSES', 'THE FIRST BOOK OF NEPHI HIS REIGN AND MINISTRY (1 Nephi)', 'THE SECOND BOOK OF NEPHI2', 'THE BOOK OF JACOB', 'THE BOOK OF MOSIAH', 'THE BOOK OF ALMA', 'THE BOOK OF HELAMAN', 'THIRD BOOK OF NEPHI', 'THE BOOK OF MORMON', 'THE BOOK OF ETHER', 'THE BOOK OF MORONI']

b = []
for i in range(len(works)):
  if i < len(works) - 1:
    start = mormon_book[10:].find(works[i])
    end = mormon_book[10:].find(works[i+1])
    b.append(mormon_book[start:end])
  else:
    start = mormon_book[10:].find(works[i])
    b.append(mormon_book[start:])

for i in range(len(b)):
  b[i] = list(filter(lambda x: len(x) >= 15, b[i].split('\n')))
  for j in range(len(b[i])):
    for k in range(len(b[i][j])):
      if b[i][j][k].isalpha():
        break
    b[i][j] = b[i][j][k:]
  b[i] = '\n'.join(b[i])
  religion_dict['author'].append(None)
  religion_dict['title'].append(works[i])
  religion_dict['text'].append(b[i])

In [ ]:
buddha_gospel = open('35895-0.txt', 'r').read()
start_idx = buddha_gospel.find('[Illustration]\n\n\n\n\n') + len('[Illustration]\n\n\n\n\n')
final_idx = buddha_gospel.find('[Names and terms must be looked up in the Glossary')
buddha_gospel = buddha_gospel[start_idx:final_idx]

religion_dict['author'].append(None)
religion_dict['title'].append('THE GOSPEL OF BUDDHA')
religion_dict['text'].append(buddha_gospel)

In [ ]:
marcus_meditations = open('pg2680.txt', 'r').read()
start_idx = marcus_meditations.find('thou have them always in a readiness.\n\n\n') + len('thou have them always in a readiness.\n\n\n')
final_idx = marcus_meditations.find('\n\n\n\nAPPENDIX')
marcus_meditations = marcus_meditations[start_idx:final_idx]

works = ['THE FIRST BOOK', 'THE SECOND BOOK', 'THE THIRD BOOK', 'THE FOURTH BOOK', 'THE FIFTH BOOK', 'THE SIXTH BOOK', 'THE SEVENTH BOOK', 'THE EIGHTH BOOK', 'THE NINTH BOOK', 'THE TENTH BOOK', 'THE ELEVENTH BOOK', 'THE TWELFTH BOOK']

b = []
for i in range(len(works)):
  if i < len(works) - 1:
    start = marcus_meditations.find(works[i])
    end = marcus_meditations.find(works[i+1])
    b.append(marcus_meditations[start:end])
  else:
    start = marcus_meditations.find(works[i])
    b.append(marcus_meditations[start:])

for i in range(len(b)):
  religion_dict['author'].append('Marcus Aurelius')
  religion_dict['title'].append(works[i])
  religion_dict['text'].append(b[i])

In [ ]:
# too hard to decipher :(
# koran = open('pg2800.txt', 'r').read()
# start_idx = koran.find('SURA1 XCVI.-THICK BLOOD, OR CLOTS OF BLOOD [I.]')
# final_idx = koran.find('\n\n\n\n\n\nEnd of The Project Gutenberg Etext of The Koran as translated by')
# koran = koran[start_idx:final_idx]

In [ ]:
data_religion = pd.DataFrame(religion_dict)

# Feature Extractor

In [ ]:
class TextFeaturizer():

  def __init__(self, text):
    self.text = text
    if len(text) > 1000000:
      self.doc = []
      for length in range(300000, len(text), 300000):
        self.doc.append(textacy.make_spacy_doc(text[length-300000:length], lang='en_core_web_lg'))
      self.doc.append(textacy.make_spacy_doc(text[length:], lang='en_core_web_lg'))
    else:
      self.doc = [textacy.make_spacy_doc(text, lang='en_core_web_lg')]
    self.n_sents =  [text_stats.n_sents(d) for d in self.doc]
    self.n_words =  [text_stats.n_words(d) for d in self.doc]

  def questions_frequency(self):
    n_sents = 0
    for ts in self.n_sents:
      n_sents += ts
    return self.text.count('?') / n_sents

  def words_per_sentence(self):
    n_words = 0
    n_sents = 0
    for i in range(len(self.doc)):
      n_words += self.n_words[i]
      n_sents += self.n_sents[i]
    return n_words / n_sents

  def ttr(self):
    """type/token ratio (TTR)"""

    stop_pos = ['ADP', 'CONJ', 'CCONJ', 'DET', 'EOL', 'PART', 'PUNCT', 'SPACE', 'SCONJ', 'SYM', 'X']
    token_count = 0
    types = set()

    for doc in self.doc:
      for token in doc:
        if not token.pos_ in stop_pos:
          types.add(token.lemma)
          token_count += 1

      if token_count == 0:
        return 0
    return len(types) / token_count

  def sttr(self, n=1000):
    """standardized type/token ratio (STTR) for every n words"""

    stop_pos = ['ADP', 'CONJ', 'CCONJ', 'DET', 'EOL', 'PART', 'PUNCT', 'SPACE', 'SCONJ', 'SYM', 'X']
    token_count = 0
    types = set()
    ttr = []
    words_read = 0

    for doc in self.doc:
      for token in doc:
        if words_read >= n:
          words_read = 0
          if token_count != 0:
            ttr.append(len(types) / token_count)
          types.clear()
          token_count = 0
        if not token.pos_ in stop_pos:
          types.add(token.lemma)
          token_count += 1
        words_read += 1

    if (words_read > n / 4 or ttr == []) and token_count != 0:
      ttr.append(len(types) / token_count)

    return sum(ttr) / len(ttr)

  def pos_freq(self):
    """ frequency of parts of speech
    """
    distribution = {}
    for pos in ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']:
      distribution[pos] = 0

    for doc in self.doc:
      for token in doc:
        distribution[token.pos_] += 1

    total_count = 0
    for pos in distribution:
      total_count += distribution[pos]

    if total_count == 0:
      return distribution

    for i in distribution:
      distribution[i] /= total_count

    return distribution

  def dep_freq(self):
    """ frequency of dependencies
    """
    distribution = {}
    for dep in DEPENDENCY_TAGS:
      distribution[dep] = 0

    for doc in self.doc:
      for token in doc:
        distribution[token.dep_] += 1

    total_count = 0
    for dep in distribution:
      total_count += distribution[dep]

    if total_count == 0:
      return distribution

    for i in distribution:
      distribution[i] /= total_count

    return distribution

  def deixis_freq(self):
    """frequency of deixis
    """

    stop_pos = ['ADP', 'CONJ', 'CCONJ', 'EOL', 'PUNCT', 'SPACE', 'SCONJ', 'SYM', 'X']
    deixis_count = 0
    token_count = 0

    for doc in self.doc:
      for token in doc:
        if token.pos_ not in stop_pos:
          token_count += 1
          if token.pos_ == "DET" and token.text.lower() not in ['the', 'a', 'an', 'no']:
            deixis_count += 1
          if token.pos_ == "PRON":
            deixis_count += 1
          if token.pos_ == "ADV" and token.text.lower() in ["here","there","now","then","soon","today","tonight","yesterday","tomorrow","above","below","behind","ahead","nearby","far","away","inside","outside","somewhere","everywhere","anywhere",]:
            deixis_count += 1

    if token_count == 0:
      return 0.0

    return deixis_count / token_count

  def verb_tense_frequency(self):
    """distribution of verb tenses
    """

    distribution = { 'Past': 0, 'Present': 0, 'Future': 0}

    for doc in self.doc:
      for i in range(len(doc)):
        token = doc[i]

        # check for cases where auxiliary verb 'will' or 'shall' is used for future
        if token.pos_ in ['AUX', 'MD']:

          if token.lemma_.lower() in ['will', 'shall']:
            distribution['Future'] += 1

        if token.pos_ == 'VERB':

          if token.tag_ in ['VBD', 'VBN']:
            distribution['Past'] += 1

          elif token.tag_ in ['VBP', 'VBZ', 'VBG']:
            distribution['Present'] += 1

          elif token.tag_ in ['VB'] and i > 1:

            # check for 'going to' future
            if doc[i-1].text.lower() == 'to' and doc[i-2].text.lower() == 'going':
              distribution['Future'] += 1

              # false alarm, gets triggered every 'going to' scenario
              distribution['Present'] -= 1

    total_count = distribution['Past'] + distribution['Present'] + distribution['Future']
    if total_count == 0:
      return distribution

    for i in distribution:
      distribution[i] /= total_count

    return distribution

  def person_frequency(self):
    """frequency of pronoun persons
    """
    distribution = {
        1: 0,
        2: 0,
        3: 0
    }

    for doc in self.doc:
      for token in doc:
        if token.pos_ == "PRON":

          if token.text.lower() in ['i', 'we', 'me', 'my', 'mine', 'myself', 'us', 'our', 'ours', 'ourselves']:
            distribution[1] += 1

          if token.text.lower() in ['you', 'your', 'yours', 'yourself']:
            distribution[2] += 1

          if token.text.lower() in ['he', 'she', 'they', 'him', 'himself', 'her', 'herself', 'them', 'his', 'hers', 'their']:
            distribution[3] += 1

    total_count = distribution[1] + distribution[2] + distribution[3]
    if total_count == 0:
      return distribution

    for i in distribution:
      distribution[i] /= total_count

    return distribution


In [ ]:
def extract_features(dataset, idx=0):
  # initialize new columns for features
  text_features = ['question_frequency', 'words_per_sentence', 'ttr', 'sttr', 'deixis_frequency', 'tense_present_frequency', 'tense_past_frequency',
                   'tense_future_frequency', 'person_1_frequency', 'person_2_frequency', 'person_3_frequency', 'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ',
                   'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE'] + DEPENDENCY_TAGS

  for feature in text_features:
    dataset[feature] = 0

  for i in tqdm(range(len(dataset))):

    featurizer = TextFeaturizer(dataset.iloc[i]['text'])
    dataset.loc[idx+i,'question_frequency'] = featurizer.questions_frequency()
    dataset.loc[idx+i,'words_per_sentence'] = featurizer.words_per_sentence()
    dataset.loc[idx+i,'ttr'] = featurizer.ttr()
    dataset.loc[idx+i,'sttr'] = featurizer.sttr(n=100)
    dataset.loc[idx+i,'deixis_frequency'] = featurizer.deixis_freq()

    tense_freq = featurizer.verb_tense_frequency()
    dataset.loc[idx+i,'tense_present_frequency'] = tense_freq['Present']
    dataset.loc[idx+i,'tense_past_frequency'] = tense_freq['Past']
    dataset.loc[idx+i,'tense_future_frequency'] = tense_freq['Future']

    person_freq = featurizer.person_frequency()
    dataset.loc[idx+i,'person_1_frequency'] = person_freq[1]
    dataset.loc[idx+i,'person_2_frequency'] = person_freq[2]
    dataset.loc[idx+i,'person_3_frequency'] = person_freq[3]

    pos_freq = featurizer.pos_freq()
    for pos in pos_freq:
      dataset.loc[idx+i,pos] = pos_freq[pos]

    dep_freq = featurizer.dep_freq()
    for dep in dep_freq:
      dataset.loc[idx+i, dep] = dep_freq[dep]


# Extract features for downloaded dataset

In [ ]:
idx = 0
for i in range(0):
  idx += len(data_books_list[i])

for i in range(0, len(data_books_list)):
  extract_features(data_books_list[i], idx)
  data_books_list[i].to_csv('data_books' + str(i) + '.csv')
  files.download('data_books' + str(i) + '.csv')
  idx += len(data_books_list[i])

In [ ]:
books = []
for i in range(86):
  books.append(pd.read_csv('data_books' + str(i) + '.csv'))
  books[-1].drop(columns=['Unnamed: 0', 'text', 'genres', 'poetry', 'religion', 'philosophy', 'historical', 'classics', 'humor', 'politics', 'short-stories', 'read-for-school', 'roman', 'non-fiction', 'history', '21st-century', 'literature', 'novels', 'fiction', 'horror', 'fantasy', 'speculative-fiction', 'science-fiction', 'supernatural', 'paranormal', '20th-century', 'dark', 'adult', 'mystery', 'adult-fiction', 'feminism', 'memoir', 'family', 'literary-fiction', 'college', 'high-school', 'american', 'mythology', 'biography', 'school', 'contemporary', 'historical-fiction', 'essays', 'coming-of-age', 'vampires', 'romance', 'novella', 'lgbt', 'thriller', 'unfinished', 'mystery-thriller', 'suspense', 'crime', 'science', 'theology', 'travel', 'adventure', 'relationships', 'cookbooks', 'reference', 'childrens', 'graphic-novels', 'comics', 'picture-books', 'action', 'magic', 'urban-fantasy', 'love', 'young-adult', 'paranormal-romance', 'teen', 'plays', 'dystopia', 'christian', 'education', 'psychology', 'economics', 'spirituality', 'writing', 'drama', 'art', 'middle-grade', 'christmas', 'chick-lit', 'new-adult', 'war', 'business', 'sociology', 'amazon', 'comedy', 'manga', 'erotica', 'animals', 'modern', 'self-help', 'realistic-fiction', 'historical-romance', 'sports', 'contemporary-romance', 'romantic-suspense', 'music', 'death', 'bdsm', 'lang'], inplace = True)
data_books = pd.concat(books, axis=0)
data_books.reset_index(drop=True, inplace=True)
data_books.drop(2014, inplace=True)
data_books.reset_index(drop=True, inplace=True)
data_books.to_csv('data_books.csv')

In [ ]:
idx = 0
for i in range(0):
  idx += len(data_shortstory_list[i])

for i in range(0, len(data_shortstory_list)):
  extract_features(data_shortstory_list[i], idx)
  #data_shortstory_list[i].to_csv('data_shortstory' + str(i) + '.csv')
  #files.download('data_shortstory' + str(i) + '.csv')
  idx += len(data_shortstory_list[i])

In [ ]:
shorts = []
for i in range(10):
  shorts.append(pd.read_csv('data_shortstory' + str(i) + '.csv'))
  shorts[-1].drop(columns=['Unnamed: 0', 'text'], inplace = True)
data_shorts = pd.concat(shorts, axis=0)
data_shorts.reset_index(drop=True, inplace=True)
data_shorts.to_csv('data_shortstory.csv')

In [ ]:
extract_features(data_news)
data_news.to_csv('data_news.csv')
files.download('data_news.csv')

In [ ]:
extract_features(data_poems)
data_poems.to_csv('data_poems.csv')
files.download('data_poems.csv')

In [ ]:
extract_features(data_recipes)
data_recipes.to_csv('data_recipes.csv')
files.download('data_recipes.csv')

In [ ]:
extract_features(data_ted)
data_ted.to_csv('data_ted.csv')
files.download('data_ted.csv')

In [ ]:
extract_features(data_conversations)
data_conversations.to_csv('data_conversations.csv')
files.download('data_conversations.csv')

In [ ]:
extract_features(data_wine)
data_wine.to_csv('data_wine.csv')
files.download('data_wine.csv')

In [ ]:
extract_features(data_religion)
data_religion.to_csv('data_religion.csv')
files.download('data_religion.csv')

In [ ]:
f = open("chatgpt_datasets_output.txt", "r")
chatgpt_dataset_dict = {'text': []}

line = f.readline()
while line:
  chatgpt_dataset_dict['text'].append(line)
  line = f.readline()

chatgpt_dataset = pd.DataFrame(chatgpt_dataset_dict)
extract_features(chatgpt_dataset)
chatgpt_dataset.to_csv('data_3story.csv')

100%|██████████| 2579/2579 [03:36<00:00, 11.89it/s]


In [ ]:
f = open("comb_datasets_output.txt", "r")
comb_dataset_dict = {'text': []}

line = f.readline()
while line:
  comb_dataset_dict['text'].append(line)
  line = f.readline()

comb_dataset = pd.DataFrame(comb_dataset_dict)
extract_features(comb_dataset)
comb_dataset.to_csv('data_3story_wp.csv')

100%|██████████| 99/99 [00:08<00:00, 12.04it/s]


In [ ]:
f = open("uniquechatgpt_datasets_output.txt", "r")
unique_chatgpt_dataset_dict = {'text': []}

line = f.readline()
while line:
  unique_chatgpt_dataset_dict['text'].append(line)
  line = f.readline()

unique_chatgpt_dataset = pd.DataFrame(unique_chatgpt_dataset_dict)
extract_features(unique_chatgpt_dataset)
unique_chatgpt_dataset.to_csv('data_1story.csv')

100%|██████████| 100/100 [00:08<00:00, 12.10it/s]


In [ ]:
f = open("writingprompts_datasets_output.txt", "r")
wp_dataset_dict = {'text': []}

line = f.readline()
while line:
  wp_dataset_dict['text'].append(line)
  line = f.readline()

wp_dataset = pd.DataFrame(wp_dataset_dict)
extract_features(wp_dataset)
wp_dataset.to_csv('data_wp.csv')

100%|██████████| 100/100 [00:08<00:00, 12.16it/s]


In [ ]:
chatgpt_dataset['method'] = 'ChatGPT3Stories'
comb_dataset['method'] =  'ChatGPT3Stories + WritingPrompts'
unique_chatgpt_dataset['method'] = 'ChatGPT1Story'
wp_dataset['method'] = 'WritingPrompts'

In [ ]:
data_cols = ['ChatGPT3Stories', 'ChatGPT3Stories + WritingPrompts', 'ChatGPT1Story', 'WritingPrompts']
feature_cols = ['question_frequency', 'words_per_sentence', 'sttr', 'deixis_frequency', 'tense_present_frequency', 'tense_past_frequency', 'tense_future_frequency', 'person_1_frequency', 'person_2_frequency', 'person_3_frequency', 'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE', 'ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']
pos_cols = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB']
dep_cols = ['ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']

final_dataset = pd.concat([wp_dataset, unique_chatgpt_dataset, chatgpt_dataset, comb_dataset]).reset_index(drop=True)
stat_dict = {'dataset' : []}
for feat_col in feature_cols:
  stat_dict[feat_col + '_mean'] = []
  stat_dict[feat_col + '_median'] = []
  stat_dict[feat_col + '_std'] = []

for col in data_cols:
  stat_dict['dataset'].append(col)
  for feat_col in feature_cols:
    stat_dict[feat_col + '_mean'].append(final_dataset.loc[final_dataset['method'] == col][feat_col].mean())
    stat_dict[feat_col + '_median'].append(final_dataset.loc[final_dataset['method'] == col][feat_col].median())
    stat_dict[feat_col + '_std'].append(final_dataset.loc[final_dataset['method'] == col][feat_col].std())

In [ ]:
data_cols = ['ChatGPT3Stories', 'ChatGPT3Stories + WritingPrompts', 'ChatGPT1Story', 'WritingPrompts']
feature_cols = ['question_frequency', 'words_per_sentence', 'sttr', 'deixis_frequency', 'tense_present_frequency', 'tense_past_frequency', 'tense_future_frequency', 'person_1_frequency', 'person_2_frequency', 'person_3_frequency', 'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE', 'ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']
pos_cols = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB']
dep_cols = ['ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']

stat_dict = {'dataset' : []}
for feat_col in feature_cols:
  stat_dict[feat_col + '_mean'] = []
  stat_dict[feat_col + '_median'] = []
  stat_dict[feat_col + '_std'] = []
for feat_col in feature_cols:
    stat_dict[feat_col + '_mean'].append(chatgpt_dataset[feat_col].mean())
    stat_dict[feat_col + '_median'].append(chatgpt_dataset[feat_col].median())
    stat_dict[feat_col + '_std'].append(chatgpt_dataset[feat_col].std())


In [ ]:
final_dataset = {'text': [], 'method':[]}
for filename in ['GPT2+ChtGPT.txt', 'GPT2-WP+ChatGPT.txt', 'last_hope_GAN_ChatGPT.txt']:
  f = open(filename, "r")

  line = f.readline()
  while line:
    final_dataset['text'].append(line)
    final_dataset['method'].append(filename)
    line = f.readline()
  f.close()
final_dataset = pd.DataFrame(final_dataset)
extract_features(final_dataset)

100%|██████████| 420/420 [00:30<00:00, 13.77it/s]


In [ ]:
data_cols = ['GPT2+ChtGPT.txt', 'GPT2-WP+ChatGPT.txt', 'last_hope_GAN_ChatGPT.txt']
feature_cols = ['question_frequency', 'words_per_sentence', 'sttr', 'deixis_frequency', 'tense_present_frequency', 'tense_past_frequency', 'tense_future_frequency', 'person_1_frequency', 'person_2_frequency', 'person_3_frequency', 'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE', 'ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']
pos_cols = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB']
dep_cols = ['ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']

stat_dict = {'dataset' : []}
for feat_col in feature_cols:
  stat_dict[feat_col + '_mean'] = []
  stat_dict[feat_col + '_median'] = []
  stat_dict[feat_col + '_std'] = []
for col in data_cols:
  stat_dict['dataset'].append(col)
  for feat_col in feature_cols:
    stat_dict[feat_col + '_mean'].append(final_dataset.loc[final_dataset['method'] == col][feat_col].mean())
    stat_dict[feat_col + '_median'].append(final_dataset.loc[final_dataset['method'] == col][feat_col].median())
    stat_dict[feat_col + '_std'].append(final_dataset.loc[final_dataset['method'] == col][feat_col].std())

In [ ]:
pd.concat([wp_dataset, unique_chatgpt_dataset, chatgpt_dataset, comb_dataset]).reset_index(drop=True).to_csv("Report3_Outputs.csv")

In [ ]:
f = open("wp_target-Copy.txt", "r")
chatgpt_dataset_dict = {'text': []}

line = f.readline()
c = 0
while line:
  c += 1
  if c == 1000:
    break
  chatgpt_dataset_dict['text'].append(line)
  line = f.readline()

chatgpt_dataset = pd.DataFrame(chatgpt_dataset_dict)
extract_features(chatgpt_dataset)
chatgpt_dataset.to_csv('data_chatgpt.csv')

100%|██████████| 999/999 [01:55<00:00,  8.66it/s]


# Extract Features for All Model Outputs

In [ ]:
import os

directory_path = 'drive/MyDrive/OutputsVariousModels/'
directory_files = os.listdir(directory_path)

for file in directory_files:
  if 'last_hope_GAN' not in file and 'nextlevelbaseline' not in file and 'GAN_baseline' not in file:
    continue
  if '.csv' in file:
    continue
  f = open(directory_path + file, "r")
  line_dict = {'text': None}
  line_dict['text'] = f.readlines()
  line_df = pd.DataFrame(line_dict)
  extract_features(line_df)
  line_df.to_csv(directory_path + file[:-4] + '.csv')

100%|██████████| 200/200 [00:13<00:00, 14.44it/s]


# Get phrase repetitions

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
f = open("wp_target-Copy.txt", "r")
max_phrase_length = 7
phrases = {}

line = f.readline()
while line:
  phrase_set = set()
  doc = nlp(line)
  tokens = [token.text.lower() for token in doc]
  for k in range(1, max_phrase_length+1):
    for j in range(len(tokens) - k):
      phrase_set.add(tuple(tokens[j:j+k]))

  for p in phrase_set:
    if p in phrases:
      phrases[p] += 1
    else:
      phrases[p] = 1

  line = f.readline()

In [ ]:
sorted_phrases = sorted(phrases.items(), key=lambda x:x[1], reverse=True)

In [ ]:
for phrase in sorted_phrases:
  if len(phrase[0]) > 3 and phrase[1] > 450:
    print(phrase)

(('once', 'upon', 'a', 'time'), 4045)
(('upon', 'a', 'time', ','), 3884)
(('once', 'upon', 'a', 'time', ','), 3883)
(('>', 'one', 'day', ','), 2985)
(('in', 'the', 'end', ','), 2956)
((',', 'there', 'was', 'a'), 2952)
(('.', 'it', 'was', 'a'), 2775)
(('a', 'time', ',', 'there'), 2289)
(('upon', 'a', 'time', ',', 'there'), 2288)
(('once', 'upon', 'a', 'time', ',', 'there'), 2287)
(('time', ',', 'there', 'was'), 2180)
(('>', 'at', 'first', ','), 2174)
(('a', 'time', ',', 'there', 'was'), 2160)
(('upon', 'a', 'time', ',', 'there', 'was'), 2159)
(('once', 'upon', 'a', 'time', ',', 'there', 'was'), 2158)
(('>', 'in', 'the', 'end', ','), 2156)
(('>', 'in', 'the', 'end'), 2156)
(('time', ',', 'there', 'was', 'a'), 2067)
(('a', 'time', ',', 'there', 'was', 'a'), 2059)
(('upon', 'a', 'time', ',', 'there', 'was', 'a'), 2058)
(('could', "n't", 'help', 'but'), 1822)
(('>', 'from', 'that', 'day'), 1522)
(('from', 'that', 'day', 'on'), 1312)
(('from', 'that', 'day', 'on', ','), 1309)
(('that', 'day'

In [ ]:
sorted_phrases[:9]

[((',',), 14348),
 (('.',), 14348),
 (('the',), 14343),
 (('to',), 14340),
 (('and',), 14340),
 (('a',), 14338),
 (('>',), 14309),
 (('of',), 14279),
 (('in',), 14213)]